# Without standard Scaler

In [1]:
import pandas, numpy
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
dataframe = pandas.read_csv("datasets/sonar.csv", header=None)
dataset = dataframe.values

In [4]:
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [5]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [6]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer="normal", bias_initializer="normal", activation="relu"))
    model.add(Dense(1, kernel_initializer="normal", bias_initializer="normal", activation="sigmoid"))
    # Compile model
    model.compile(loss="binary_crossentropy" , optimizer="adam" ,metrics=["accuracy"])
    return model

In [7]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 80.71% (9.27%)


In [8]:
create_baseline().summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 60)                3660      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 61        
Total params: 3,721
Trainable params: 3,721
Non-trainable params: 0
_________________________________________________________________


# With stanadard scaler

In [14]:
from sklearn.pipeline import Pipeline
numpy.random.seed(seed)
estimators = []
estimators.append(("std", StandardScaler()))
estimators.append(("clf", KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.57% (6.88%)


# Small network

In [21]:
# small network
def create_small():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, kernel_initializer='normal', bias_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', bias_initializer='normal', activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [18]:
create_small().summary()

Model: "sequential_71"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_142 (Dense)            (None, 30)                1830      
_________________________________________________________________
dense_143 (Dense)            (None, 1)                 31        
Total params: 1,861
Trainable params: 1,861
Non-trainable params: 0
_________________________________________________________________


In [17]:
numpy.random.seed(seed)
estimators_small = []
estimators_small.append(('std', StandardScaler()))
estimators_small.append(('mlp', KerasClassifier(build_fn=create_small, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators_small)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 83.62% (7.03%)


# Large network

In [20]:
# Large network
def create_large():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer='normal', bias_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', bias_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', bias_initializer='normal', activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [22]:
create_large().summary()

Model: "sequential_72"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_144 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_145 (Dense)            (None, 30)                1830      
_________________________________________________________________
dense_146 (Dense)            (None, 1)                 31        
Total params: 5,521
Trainable params: 5,521
Non-trainable params: 0
_________________________________________________________________


In [23]:
estimators_large = []
estimators_large.append(('std', StandardScaler()))
estimators_large.append(('mlp', KerasClassifier(build_fn=create_large, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators_large)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.57% (6.88%)
